In [5]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request

In [6]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = "1UZ6ZI3MWDeSO_w1joJPHb8_mgzRuL4OSAgpRmkjFjyo"
RANGE_NAME = "Dataset"

In [3]:
# Update which one is in the Tree

In [4]:
# Initialize credentials
credentials = None
if os.path.exists("token.json"):
    credentials = Credentials.from_authorized_user_file("token.json", SCOPES)
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credits.json", SCOPES)
        credentials = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
        token.write(credentials.to_json())

try:
    # Build the service
    service = build("sheets", "v4", credentials=credentials)
    sheet = service.spreadsheets()

    # Call the Sheets API to fetch data
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values or len(values) < 2:
        print('No data found or only header row present.')
    else:
        # Skip the first row (header) and process the remaining rows
        values_to_process = values[1:]  # Exclude the header row

        # Process rows to find longest sequences for the same values in Columns C and B
        records = {}  # key: (C, B) values, value: (index, sequence length)
        for index, row in enumerate(values_to_process, start=1):  # Start index at 1 to adjust for the header
            if len(row) > max(2, 1, 16):  # Ensure row has enough columns
                key = (row[2], row[1])  # Column C and B values
                seq_length = len(row[16])  # Column Q value (sequence length)
                if key not in records or records[key][1] < seq_length:
                    records[key] = (index, seq_length)

        # Update values to include "Yes" or "No" in Column G based on the longest sequence
        for key, (winning_index, _) in records.items():
            for index, row in enumerate(values_to_process, start=1):  # Adjust index to match overall list
                if len(row) > max(2, 1, 16) and (row[2], row[1]) == key:
                    if index == winning_index:
                        if len(row) < 6:  # Ensure the row has a Column G
                            row.extend([""] * (6 - len(row)))
                        row[6] = "Yes"  # Mark the row with the longest sequence
                    else:
                        if len(row) < 6:
                            row.extend([""] * (6 - len(row)))
                        row[6] = "No"

        # Prepare the data for updating (including the header row)
        body = {
            'values': values  # Use the original values list to maintain the header row
        }
        # Update the sheet with "Yes" or "No" in Column G
        update_result = sheet.values().update(
            spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME,
            valueInputOption="RAW", body=body).execute()
        print(f"{update_result.get('updatedCells')} cells updated.")

except HttpError as error:
    print(error)

1124 cells updated.


In [22]:
# extract the relevant sequences with Timless and Timeout

In [7]:
# Initialize credentials
credentials = None
if os.path.exists("token.json"):
    credentials = Credentials.from_authorized_user_file("token.json", SCOPES)
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credits.json", SCOPES)
        credentials = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
        token.write(credentials.to_json())

try:
    # Build the service
    service = build("sheets", "v4", credentials=credentials)
    sheet = service.spreadsheets()

    # Call the Sheets API to fetch data
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        # Open or create a FASTA file for writing sequences marked with "Yes" in column G
        with open("Output/sequences_Timeout_and_timeless.fasta", "w") as fasta_file:
            for row in values:
                if len(row) >= 7 and row[6] == "Yes":  # Check if column G (index 6) is "Yes"
                    # Assuming the name is in column B (index 1), C (index 2), and F (index 5) and the sequence in column Q (index 16)
                    species = row[1] if len(row) > 1 else "Unknown"
                    gene = row[2] if len(row) > 2 else "Unknown"
                    protein_id = row[5] if len(row) > 5 else "Unknown"
                    sequence = row[16] if len(row) > 16 else "Unknown"
                    # Write to FASTA format
                    fasta_file.write(f">{species}|{gene}|{protein_id}\n{sequence}\n")

except HttpError as error:
    print(error)

In [24]:
# extract the relevant sequences Only Timeout

In [25]:
# Initialize credentials
credentials = None
if os.path.exists("token.json"):
    credentials = Credentials.from_authorized_user_file("token.json", SCOPES)
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        credentials.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credits.json", SCOPES)
        credentials = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
        token.write(credentials.to_json())

try:
    # Build the service
    service = build("sheets", "v4", credentials=credentials)
    sheet = service.spreadsheets()

    # Call the Sheets API to fetch data
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        # Open or create a FASTA file for writing sequences marked with "Yes" in column G
        with open("Output/sequences_Timeout.fasta", "w") as fasta_file:
            for row in values:
                if len(row) >= 7 and row[6] == "Yes" and row[2] == "Timeout":  # Check if column G (index 6) is "Yes"
                    # Assuming the name is in column B (index 1), C (index 2), and F (index 5) and the sequence in column Q (index 16)
                    species = row[1] if len(row) > 1 else "Unknown"
                    gene = row[2] if len(row) > 2 else "Unknown"
                    protein_id = row[5] if len(row) > 5 else "Unknown"
                    sequence = row[16] if len(row) > 16 else "Unknown"
                    # Write to FASTA format
                    fasta_file.write(f">{species}|{gene}|{protein_id}\n{sequence}\n")

except HttpError as error:
    print(error)